[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/weaviate/recipes/blob/main/weaviate-features/multi-tenancy/multi-tenant-academy-course.ipynb)

#  Multi-tenant operations

This notebook walks through the Weaviate Academy [Multi-tenancy course](https://weaviate.io/developers/academy/py/multitenancy).

### Edit the docker-compose configuration

1. Create a `.env` file in the directory where you run `docker compose up -d`.
2. Add your AWS credentials to the `.env` file.
3. Add variables for the credentials to `docker-compose.yml`

##### .env file

```shell
AWS_ACCESS_KEY=ENTER-YOUR-AWS-KEY-HERE
AWS_ACCESS_SECRET_KEY=ENTER-YOUR-AWS-SECRET-KEY-HERE
AWS_REGION=YOUR-AWS-REGION-CODE-HERE
```

To confirm the variable substitution, run `docker compose config`.

Start your Weaviate instance, `docker compose up -d`.

##### docker-config.yml

```shell
---
name: "multi_tenant_demo_instance"
services:
  weaviate_anon:
    command:
    - --host
    - 0.0.0.0
    - --port
    - '8080'
    - --scheme
    - http
    image: cr.weaviate.io/semitechnologies/weaviate:1.26.3 # Update if needed
    ports:
    - 8080:8080
    - 50051:50051
    restart: on-failure:0
    environment:
      QUERY_DEFAULTS_LIMIT: 25
      AUTHENTICATION_ANONYMOUS_ACCESS_ENABLED: 'true'
      PERSISTENCE_DATA_PATH: '/var/lib/weaviate'
      DEFAULT_VECTORIZER_MODULE: 'none'
      ENABLE_API_BASED_MODULES: 'true'
      ASYNC_INDEXING: 'true'
      ENABLE_MODULES: 'backup-filesystem,offload-s3'
      AWS_REGION: ${AWS_REGION}
      AWS_ACCESS_KEY_ID: ${AWS_ACCESS_KEY}
      AWS_SECRET_ACCESS_KEY: ${AWS_ACCESS_SECRET_KEY}
      OFFLOAD_S3_BUCKET: 'PUT_YOUR_S3_BUCKET_NAME_HERE'
      OFFLOAD_S3_BUCKET_AUTO_CREATE: 'true'
      BACKUP_FILESYSTEM_PATH: '/var/lib/weaviate/backups'
      CLUSTER_HOSTNAME: 'node1'
...
```

### Install the Weaviate Python v4 client

In [ ]:
#!which python

# Uncomment to clear your current pip cache
# !pip cache purge

# Uncomment to upgrade pip
# !pip install --upgrade pip

# Install client from public released
!pip3 install --no-cache -U "weaviate-client==4.*"

# Check installed client version
!pip show weaviate-client | grep Version


### Create a client object

In [ ]:
import weaviate, os
from weaviate.classes.config import Configure, Property, DataType

# Connect to a WCS instance
client = weaviate.connect_to_local(
    headers={
        "X-Cohere-Api-Key": os.environ["COHERE_API_KEY"],
    }
)

# Uncomment to check the client connection
print(client.is_ready())

### Create a collection

In [140]:
collection_name = "MultiTenantDemo"

# Uncomment to delete data from prior runs
if (client.collections.exists(collection_name)):
    client.collections.delete(collection_name)

# Create the collection
multi_tenant_collection = client.collections.create(
    name=collection_name,
    multi_tenancy_config=Configure.multi_tenancy(
        enabled=True,
        auto_tenant_creation=True,
        auto_tenant_activation=True,
    ),
    properties=[
        Property(name="text", data_type=DataType.TEXT),
        Property(name="date", data_type=DataType.DATE),
        Property(name="tags", data_type=DataType.TEXT_ARRAY),
    ],
    vectorizer_config=[
        Configure.NamedVectors.text2vec_cohere(
            name="text",
            source_properties=["text"],
            vector_index_config=Configure.VectorIndex.dynamic(
                hnsw=Configure.VectorIndex.hnsw(
                    quantizer=Configure.VectorIndex.Quantizer.sq(training_limit=50000)
                ),
                flat=Configure.VectorIndex.flat(
                    quantizer=Configure.VectorIndex.Quantizer.bq()
                ),
                threshold=10000
            )
        )
    ],
    generative_config=Configure.Generative.cohere(model="command-r-plus")
)


### Confirm that the collection exists

In [ ]:
response = client.collections.list_all()

for r in response:
    print(r)


### Get collection object

In [143]:
multi_tenant_collection = client.collections.get(collection_name)

### Create a tenant

In [142]:
from weaviate.classes.tenants import Tenant

multi_tenant_collection.tenants.create(
    tenants=[
        Tenant(name="steve85"),
        Tenant(name="bobby84"),
    ]
)

# # Uncomment to list the tenants
# tenants = multi_tenant_collection.tenants.get()
# print(tenants)

### Add data to one tenant

In [144]:
from datetime import datetime, timezone

# Get collection object then specify the tenant
multi_tenant_collection = client.collections.get(collection_name)
tenant = multi_tenant_collection.with_tenant("steve85")

insert_result = tenant.data.insert(
    properties={
        "text": "What amazing food we had at Quay! It was totally worth it.",
        "date": datetime(2024, 5, 15).replace(tzinfo=timezone.utc),
        "tags": ["restaurant", "experience"],
    }
)

# # Uncomment to see the UUID of the inserted object
# print(insert_result)

### Batch import data to a single tenant

In [145]:
# Some sample data
journal_entries = [
     {"text": "Loved it", "date": datetime(2024, 3, 4).replace(tzinfo=timezone.utc), "tags": ["restaurant", "experience"]},
     {"text": "Hated it", "date": datetime(2024, 4, 5).replace(tzinfo=timezone.utc), "tags": ["cafe"]},
     {"text": "Meh", "date": datetime(2024, 5, 6).replace(tzinfo=timezone.utc), "tags": ["dinner", "group"]}
    ]

# Get collection object then specify the tenant
multi_tenant_collection = client.collections.get(collection_name)
tenant = multi_tenant_collection.with_tenant("steve85")

batch_results = []
with tenant.batch.fixed_size(100) as batch:
    for journal_entry in journal_entries:
        batch_results.append(batch.add_object(journal_entry))

# # Uncomment to see the UUIDs of the inserted objects
# print(batch_results)

### Check the number of objects

In [ ]:
# Get the collection object and the tenant
multi_tenant_collection = client.collections.get(collection_name)
tenant = multi_tenant_collection.with_tenant("steve85")

response = tenant.aggregate.over_all(total_count=True)

# print(response)               # All info
print(response.total_count)   # limited info


### Automatically create a tenant

In [127]:
# Get the collection object
multi_tenant_collection = client.collections.get(collection_name)

# Specify a tenant that doesn't exist yet
new_tenant = multi_tenant_collection.with_tenant("yana42")

# # Uncomment to list the number of tenants before inserting
# tenants = multi_tenant_collection.tenants.get()
# print(f"Before: {len(tenants)}")

insert_data_result = new_tenant.data.insert({
    "date": datetime(2024, 7, 7).replace(tzinfo=timezone.utc),
    "tags": ["events", "grand prix"],
    "text": "La nacional rocks",
})

# # Uncomment to see the UUID of the inserted object
# print(insert_data_rt)

# # Uncomment to list the number of tenants after inserting
# tenants = multi_tenant_collection.tenants.get()
# print(f"After: {len(tenants)}")

# # Uncomment to list the tenants
# tenants = multi_tenant_collection.tenants.get()
# for t in tenants:
#     print(t)

### Run a query

In [ ]:
from weaviate.classes.query import Filter
from datetime import datetime, timezone

# Get the collection object and the tenant
multi_tenant_collection = client.collections.get(collection_name)
tenant = multi_tenant_collection.with_tenant("steve85")

start_date = datetime(2024, 1, 1).replace(tzinfo=timezone.utc)
end_date = datetime(2024, 4, 15).replace(tzinfo=timezone.utc)

# Find the entries between two dates
response = tenant.query.fetch_objects(
    filters=(
        Filter.by_property("date").greater_or_equal(start_date) &
        Filter.by_property("date").less_or_equal(end_date)
    ),
    limit=10
)

for obj in response.objects:
    print(obj.properties)

### Run a hybrid query

In [ ]:
# Get the collection object and the tenant
multi_tenant_collection = client.collections.get(collection_name)
tenant = multi_tenant_collection.with_tenant("steve85")

response = tenant.query.hybrid(
    query="great food",
    limit=2
)

for obj in response.objects:
    print(obj.properties)

### Check tenant status

In [ ]:
tenants = multi_tenant_collection.tenants.get()

# Check all tenants
for t in tenants:
    print(f"name: {tenants[t].name} status: {tenants[t].activityStatus}")

# # Uncomment to check a single tenant
# tenant_to_check = 'steve85'
# print(f"name: {tenants[tenant_to_check].name} status: {tenants[tenant_to_check].activityStatus}")


### Deactivate a tenant

In [ ]:
from weaviate.classes.tenants import Tenant, TenantActivityStatus

# Select a tenant to update
tenant_to_update = 'yana21'

# Create the tenant
new_tenant = multi_tenant_collection.with_tenant(tenant_to_update)
insert_data_result = new_tenant.data.insert({
    "date": datetime(2024, 7, 7).replace(tzinfo=timezone.utc),
    "tags": ["events", "grand prix"],
    "text": "La nacional rocks",
})

# Check the initial status
tenants = multi_tenant_collection.tenants.get()
print(f"BEFORE: name: {tenants[tenant_to_update].name} status: {tenants[tenant_to_update].activityStatus}")

multi_tenant_collection.tenants.update(
    Tenant(name=tenant_to_update, activity_status=TenantActivityStatus.INACTIVE)
)

# Check the updated status
tenants = multi_tenant_collection.tenants.get()
print(f"AFTER: name: {tenants[tenant_to_update].name} status: {tenants[tenant_to_update].activityStatus}")

# # Uncomment to reset the status to ACTIVE
# multi_tenant_collection.tenants.update(
#     Tenant(name=tenant_to_update, activity_status=TenantActivityStatus.ACTIVE)
# )
# print(f"REACTIVATED: name: {tenants[tenant_to_update].name} status: {tenants[tenant_to_update].activityStatus}")

### Remove a tenant

In [ ]:
multi_tenant_collection = client.collections.get(collection_name)

# Select a tenant to remove
# NOTE: As of v1.26.3, the tenant status must not be COLD
tenant_to_remove = 'sonia77'

# Create the example tenant
new_tenant = multi_tenant_collection.with_tenant(tenant_to_remove)
new_tenant.data.insert({
    "date": datetime(2024, 7, 7).replace(tzinfo=timezone.utc),
    "tags": ["events", "grand prix"],
    "text": "La nacional rocks",
})

# Uncomment to check the initial tenant status
tenants = multi_tenant_collection.tenants.get()
print(f"BEFORE: name: {tenants[tenant_to_remove].name} status: {tenants[tenant_to_update].activityStatus}\n")

# Caution - this action removes all data for the deleted tenant(s)
# multi_tenant_collection.tenants.remove([tenant_to_remove])  # Use a list to remove multiple tenants
multi_tenant_collection.tenants.remove(tenant_to_remove)

# Uncomment to check tenant status
tenants = multi_tenant_collection.tenants.get()
for t in tenants:
    print(f"AFTER: name: {tenants[t].name} status: {tenants[t].activityStatus}")


### Move a tenant to S3

In [ ]:
from weaviate.classes.tenants import Tenant, TenantActivityStatus

multi_tenant_collection = client.collections.get(collection_name)

tenant_to_offload = 'carlos44'

# Uncomment to create the example tenant
new_tenant = multi_tenant_collection.with_tenant(tenant_to_offload)
insert_result = new_tenant.data.insert({
    "date": datetime(2024, 7, 7).replace(tzinfo=timezone.utc),
    "tags": ["events", "grand prix"],
    "text": "La nacional rocks",
})

# # Uncomment to see the UUID of the inserted item
# print(insert_result)

# multi_tenant_collection.tenants.update(
#     Tenant(name=tenant_to_offload, activity_status=TenantActivityStatus.INACTIVE)
# )
tenants = multi_tenant_collection.tenants.get()
for t in tenants:
    print(f"name: {tenants[t].name} status: {tenants[t].activityStatus}")

multi_tenant_collection.tenants.update(
    Tenant(name=tenant_to_offload, activity_status=TenantActivityStatus.OFFLOADED)
)

# Uncomment to see the status of the tenants
tenants = multi_tenant_collection.tenants.get()
for t in tenants:
    print(f"name: {tenants[t].name} status: {tenants[t].activityStatus}")
